In [ ]:
cog_key = "COPY_THE_KEY_HERE"
cog_endpoint = "COPY_THE_ENDPOINT_URL_HERE"

print("ready to use cognitive service")

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient 
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
import time
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as pyplot 
from PIL import Image, ImageDraw
import os
%matplotlib inline

#get a client for the compouter vision service 
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

#read the handwritten image file
image_path = os.path.join('images', 'hand.jpg')
image_stream = open(image_path, "rb")

#submit a request to read handwritten text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream, raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

#wait for the asynchronous operation to complete
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

#if the operation was successfuly, process the text line by line
if read_result.status == OperationStatusCodes.succeeded:
    for result in read_result.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

#open image and display it
print("\n")
fig = pyplot.figure(figsize=(12,12))
img = Image.open(image_path)
pyplot.axis("off")
pyplot.imshow(img)
